In [4]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor,nn
import torch.nn.functional as F
from fastcore.test import test_close
from urllib.request import urlretrieve
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)
path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

## setup


In [5]:
n,m = x_train.shape
c = y_train.max() +1
nh = 50

In [12]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]

    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [13]:
model = Model(m,nh,10)
pred = model(x_train)
pred.shape

torch.Size([50000, 10])

## Cross Entropy loss

In [14]:
def log_softmax(x): return (x.exp() / (x.exp().sum(-1,keepdim=True))).log()

In [15]:
log_softmax(pred)

tensor([[-2.12, -2.40, -2.39,  ..., -2.44, -2.18, -2.37],
        [-2.13, -2.47, -2.33,  ..., -2.48, -2.15, -2.27],
        [-2.26, -2.38, -2.47,  ..., -2.35, -2.24, -2.28],
        ...,
        [-2.21, -2.29, -2.47,  ..., -2.40, -2.23, -2.36],
        [-2.22, -2.38, -2.37,  ..., -2.41, -2.15, -2.29],
        [-2.13, -2.40, -2.48,  ..., -2.42, -2.21, -2.24]], grad_fn=<LogBackward0>)

In [16]:
def log_softmax(x): return (x - (x.exp().sum(-1,keepdim=True))).log()